<img src="../../SAR_Training/English/Master/NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Interacting with GEOGloWS ECMWF Streamflow Data for use with SAR Flood Detections</b></font>

<br>
<font size="4"> <b> Jim Nelson, BYU and Franz J Meyer; University of Alaska Fairbanks </b> <br>
<img src="../../SAR_Training/English/Master/NotebookAddons/UAFLogo_A_647.png" style="padding:5px;" width="170" align="right" /></font>

<font size="3">This notebook demonstrates how to interact with the GEOGloWS ECMWF Streamflow Forecast Database. This could be useful in the context of SAR-based flood mapping. <cite><a href="https://www.sciencedirect.com/science/article/pii/S0034425720301024"><i>Hindcast and forecast of daily inundation extents using satellite SAR and altimetry data with rotated empirical orthogonal function analysis: Case study in Tonle Sap Lake Floodplain</i></a> by Chang et al.</cite>, describes a method to utilize high temporal frequency steamflow information to temporarily interpolate SAR-based flood maps, arriving at daily flood extent information. Additionally, the streamflow information from GEOGloWS may be useful as an automatic trigger for SAR-based flood mapping subscriptions. For this task, the flood warning information from GEOGloWS might be useful.
    
The follow sections describe how GEOGLOWS data can be accessed and what kind of information is contained.

</font>
<hr>


</font>

# Introduction

<font face="Calibri" size="3">You can interact with the streamflow model using the geoglows python package. This notebook will take you through some of the functions available. For more information, please refer to https://geoglows.readthedocs.io.

Before starting or if you experience problems, find the menu on the top left of your webpage, near the Google Collaboratory or Jupyter Notebook logo.

In Collaboratory
1.   Click runtime -> reset all runtimes
2.   Click Edit -> clear all outputs

In Jupyter Notebook:
1.   Click Kernel -> Reset & Clear Output
</font>

# Installing Necessary Python Packages

<font face="Calibri" font size="3"> <b><u>!! Note: You have have to run the following cell twice !!</u></b> Occassionally, the install of geoglows doesn't succeed the first time around and the code cell needs to be run a second time to install failed components. 
</font>

In [ ]:
# Start by installing the package and importing it to your code. Run this cell to do that.
!pip install geoglows
import geoglows
from IPython.core.display import display, HTML

In [ ]:
import geoglows
from IPython.core.display import display, HTML

# Searching GEOGloWS by latitude and longitude

There are many options to interact with the API, you can search with latitude and longitude for anywhere on the planet.

In [ ]:
# Example in Mexico
#latitude = 22.824093 
#longitude = -98.785892

# Example in Peru
#latitude = -4.424 
#longitude = -77.164

# Example in Bangladesh
latitude = 24.876 
longitude = 91.89

In [ ]:
# Want to check where this stream is? Run this cell to get a link to open a google maps page zoomed to this location!
print('https://www.google.com/maps/place/' + str(latitude) + ',' + str(longitude))

In [ ]:
# This cell gets some basic forecast data and return period data with the latitude and longitude from above
forecasts = geoglows.streamflow.forecast_stats(False, lat=latitude, lon=longitude)
return_periods = geoglows.streamflow.return_periods(False, lat=latitude, lon=longitude)

In [ ]:
# This cell turns the forecast data and return periods information into a plot
forecast_figure = geoglows.streamflow.forecast_plot(forecasts, return_periods)
forecast_figure.show()

# Interface with a reach_id

<font face="Calibri" size="3">You just turned the latitude and longitude of a stream and a couple seconds of waiting into a graph of forecasted streamflow. How cool is that!? What else can GEOGloWS do for you?

You can also speed up your process and get access to even more data if you use some information about the model. Each river on the planet has been assigned an ID for logistical purposes. You can determine the ID and then use that in place of latitude and longitude. This makes your process faster so we can get even more done!

Run the next 3 cells to turn your latitude and longitude into a reach_id and get some metadata about that spot.
</font>

In [ ]:
# Same example, But this time map the latitude and longitude to an ID used by the GEOGloWS model
mapping_data = geoglows.streamflow.latlon_to_reach(latitude, longitude)

In [ ]:
# Take a look at the kind of information thats available by interacting with the model directly.
# The distances are in degrees of latitude and longitude
mapping_data

In [ ]:
# What we're really interested in is just the identifier, so we'll pick that
reach_id = mapping_data['reach_id']

# Access even more data using your reach_id

<font face="Calibri" size="3">Now that we have an ID, we can speed up our hydrology.

Run the next cell. It requests 5 different pieces of information from the GEOGloWS model. Notice that this cell that asks for 5 pieces of information will take less time than the 2 we did earlier using only latitude and longitude.
</font>

In [ ]:
# Go collect all the data 
# endpoint = 'http://gsf-api-vm.eastus.cloudapp.azure.com/api/'
endpoint = 'https://tethys2.byu.edu/localsptapi/api/'
stats = geoglows.streamflow.forecast_stats(reach_id, endpoint=endpoint)
ensembles = geoglows.streamflow.forecast_ensembles(reach_id, endpoint=endpoint)
hist = geoglows.streamflow.historic_simulation(reach_id, endpoint=endpoint)
rperiods = geoglows.streamflow.return_periods(reach_id, endpoint=endpoint)
seasonal = geoglows.streamflow.seasonal_average(reach_id, endpoint=endpoint)
warnings = geoglows.streamflow.forecast_warnings(mapping_data['region'], endpoint=endpoint,return_format='csv',lon=longitude,lat=latitude)

# Visualizing data

<font face="Calibri" size="3">You just gathered forecasted and historical simulation streamflow data for the river you chose by latitude and longitude. Now what can we do with it? 

The geoglows package has some tools for showing all the data you can get from the GEOGloWS ECMWF Streamflow model. The easiest way to show this data is to plot it. Each of the following cells will turn the data we requested into a figure and then show it. </font>

In [ ]:
# Statistical summary of the forecasted flows
forecast_figure = geoglows.streamflow.forecast_plot(stats, rperiods, reach_id=reach_id)
forecast_figure.show()

In [ ]:
# View each of the forecasts individually
ensembles_figure = geoglows.streamflow.ensembles_plot(ensembles, rperiods, reach_id=reach_id)
ensembles_figure.show()

In [ ]:
warnings[0:15]

In [ ]:
# Historically simulated flow
historic_figure = geoglows.streamflow.historical_plot(hist, rperiods, reach_id=reach_id)
historic_figure.show()

In [ ]:
# Processing the historical data into a daily average flow
seasonal_figure = geoglows.streamflow.seasonal_plot(seasonal, reach_id=reach_id)
seasonal_figure.show()

In [ ]:
# Flow Duration Curve (derived from the historical data)
flow_duration_figure = geoglows.streamflow.flow_duration_curve_plot(hist, reach_id=reach_id)
flow_duration_figure.show()

In [ ]:
# View the probabilities table
prob_table = geoglows.streamflow.probabilities_table(stats, ensembles, rperiods)
display(HTML(prob_table))

# Version Log

<font face="Calibri" size="2" color="gray"> <i> Version 1.1 - Franz J Meyer; 5/1/2020

Recent Changes:
- Fixed API issues for grabbing reach_id
- Fixed changed API call when grabbing forecast and return periods by lat/lon
- Edited text in introduction
- Added collapsible headings features for improved notebook organization
    
For future issues, check https://colab.research.google.com/drive/1Ip4_ruZH1nzpfYGzr7v3wXtuR5kMgKdN#scrollTo=H7I9xhRMOAJI for potential recent changes to GEOGLoWS API.

</i></font>